In [0]:
%python
filepath = "/Volumes/globalretail/data_pipeline/bronze/transaction_data/transaction.snappy.parquet"
df = spark.read.parquet(filepath)
df.show()
"""Transaction Date is in String format,must change to date format """

In [0]:
from pyspark.sql.functions import to_timestamp,col
new_df = df.withColumn("transaction_date", to_timestamp(col("transaction_date")))
display(new_df)
       

In [0]:
%python
from pyspark.sql.functions import current_timestamp
final_df = new_df.withColumn("ingestion_timestamp",current_timestamp())
display(final_df)

In [0]:
%python
spark.sql("use globalretail_bronze_layer")
final_df.write.format("delta").mode("append").saveAsTable("bronze_transactions")

In [0]:
%python
spark.sql("SELECT * FROM bronze_transactions LIMIT 100").show()

In [0]:
import datetime

# Define the archive folder and file path
archive_folder = "/Volumes/globalretail/data_pipeline/bronze/transaction_data/archive/"
archive_filepath = archive_folder + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Move the file to the archive location
try:
    dbutils.fs.mv(filepath, archive_filepath)
    print(f"File moved to: {archive_filepath}")
except Exception as e:
    print(f"Error moving file: {e}")